In [1]:
import import_ipynb
from dataset import SegmentationDataset
from model import UNet
import config
import utils
from torch.nn import BCEWithLogitsLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os

importing Jupyter notebook from dataset.ipynb
importing Jupyter notebook from model.ipynb
importing Jupyter notebook from config.ipynb
importing Jupyter notebook from utils.ipynb


In [2]:
plt.style.use('ggplot')

In [3]:
imagePaths = sorted(list(paths.list_images(config.IMAGE_DATASET_PATH)))
maskPaths = sorted(list(paths.list_images(config.MASK_DATASET_PATH)))

# 依照設定的比例分割測試集和驗證集
(trainImages, testImages) = train_test_split(imagePaths, test_size=config.TEST_SPLIT, random_state=42)
(trainMasks, testMasks) = train_test_split(maskPaths, test_size=config.TEST_SPLIT, random_state=42)
print("[INFO] saving testing image paths...")
f = open(config.TEST_PATHS, "w")
f.write("\n".join(testImages))
f.close()

[INFO] saving testing image paths...


In [4]:
transforms = transforms.Compose([transforms.ToPILImage(),
    transforms.Resize((config.INPUT_IMAGE_HEIGHT,
        config.INPUT_IMAGE_WIDTH)),
    transforms.ToTensor()])

trainDS = SegmentationDataset(imagePaths=trainImages, maskPaths=trainMasks,
    transforms=transforms)
testDS = SegmentationDataset(imagePaths=testImages, maskPaths=testMasks,
    transforms=transforms)
print(f"[INFO] found {len(trainDS)} examples in the training set...")
print(f"[INFO] found {len(testDS)} examples in the test set...")

trainLoader = DataLoader(trainDS, shuffle=True,
    batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
    num_workers=0)
testLoader = DataLoader(testDS, shuffle=False,
    batch_size=config.BATCH_SIZE, pin_memory=config.PIN_MEMORY,
    num_workers=0)

[INFO] found 2414 examples in the training set...
[INFO] found 427 examples in the test set...


In [5]:
unet = UNet().to(config.DEVICE)


if os.path.exists('./output/last_model.pth'):
    unet.load_state_dict(torch.load('./output/last_model.pth')['model_state_dict'])

lossFunc = BCEWithLogitsLoss()
opt = Adam(unet.parameters(), lr=config.INIT_LR)

trainSteps = len(trainDS) // config.BATCH_SIZE
testSteps = len(testDS) // config.BATCH_SIZE
# 建立字典便於追蹤loss
H = {"train_loss": [], "test_loss": []}

In [6]:
print("[INFO] training the network...")

save_best_model = utils.SaveBestModel()
train_loss_list = []
val_loss_list = []

startTime = time.time()
for e in tqdm(range(config.NUM_EPOCHS)):
    # 每次驗證都需要轉換為eval，訓練時要轉換回來
    unet.train()
    
    totalTrainLoss = 0
    totalTestLoss = 0
    
    # training
    for (i, (images, masks)) in enumerate(trainLoader):
        (images, masks) = (images.to(config.DEVICE), masks.to(config.DEVICE))
        # forward pass
        pred = unet(images)
        # training loss
        loss = lossFunc(pred, masks)
        opt.zero_grad()
        loss.backward()
        opt.step()
        # totalloss
        totalTrainLoss += loss
    # valid
    with torch.no_grad():
        unet.eval()
        
        for (images, masks) in testLoader:
            (images, masks) = (images.to(config.DEVICE), masks.to(config.DEVICE))
            pred = unet(images)
            totalTestLoss += lossFunc(pred, masks)
            
    avgTrainLoss = totalTrainLoss / trainSteps
    avgTestLoss = totalTestLoss / testSteps
    # update training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["test_loss"].append(avgTestLoss.cpu().detach().numpy())
    
    print("[INFO] EPOCH: {}/{}".format(e + 1, config.NUM_EPOCHS))
    print("Train loss: {:.6f}, Test loss: {:.4f}".format(
        avgTrainLoss, avgTestLoss))
    
    save_best_model(
        avgTestLoss, e+1, unet, opt
    )
    utils.save_model(
        e+1, unet, opt
    )
    utils.save_loss_plot(
        config.BASE_OUTPUT, H["train_loss"],  H["test_loss"]
    )
    
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(
    endTime - startTime))

[INFO] training the network...


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

[INFO] EPOCH: 1/100
Train loss: 0.401225, Test loss: 0.8174

Best validation loss: 0.8173651099205017

Saving best model for epoch: 2



  1%|▊                                                                               | 1/100 [00:46<1:16:36, 46.43s/it]

SAVING PLOTS COMPLETE...
[INFO] EPOCH: 2/100
Train loss: 0.367271, Test loss: 0.3766

Best validation loss: 0.3765992224216461

Saving best model for epoch: 3



...

SAVING PLOTS COMPLETE...
[INFO] EPOCH: 99/100
Train loss: 0.102669, Test loss: 0.2959


 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [1:10:45<00:40, 40.47s/it]

SAVING PLOTS COMPLETE...
[INFO] EPOCH: 100/100
Train loss: 0.106046, Test loss: 0.2963


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:11:25<00:00, 42.85s/it]

SAVING PLOTS COMPLETE...
[INFO] total time taken to train the model: 4285.35s
